In [1]:
import os, glob
import json
from functions import load_txt_as_lst, read_alignments

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

# TEST: Run labse on every sents file in input directory, then write to scratch
Moved to .py file

In [17]:
sents_dir = "/home/craig.car/repos/chiron/chironata/data/src_data/"
for sents_path in glob.iglob(sents_dir+"*.sents"):
    print(os.path.splitext(os.path.basename(sents_path))[0])
    break

urn:cts:greekLit:tlg0001.tlg001


In [18]:
sents_dir = "/home/craig.car/repos/chiron/chironata/data/french_trans-dev/"
for sents_path in glob.iglob(sents_dir+"*.sents"):
    print(os.path.splitext(os.path.basename(sents_path))[0])
    break

aeschylus_1863


In [24]:
sents_dir = "/home/craig.car/repos/chiron/chironata/data/src_data/"
dir_out = "/scratch/craig.car/sents_embeds/src_embeds/"
    
for sents_path in glob.iglob(sents_dir+"*.sents"):
    filename = os.path.splitext(os.path.basename(sents_path))[0]
    if os.path.isfile(dir_out+filename+".emb") == False:
        print(filename)
        txt_lst = load_txt_as_lst(sents_path)
        # labse_embeds = build_embeddings_huggingface(txt_lst, model)
        print("built embeds")
        path_out = dir_out+filename+".emb"
        # write_to_file(labse_embeds,path_out)
        print(path_out)
    break

# Run sentence retrieval and score

In [2]:
def compute_sim_matrix_cosine(src_embeds, tgt_embeds):
    # using cosine sim
    return cosine_similarity(src_embeds, tgt_embeds)

In [3]:
# load lookup table
lookup_path = "/home/craig.car/repos/chiron/chironata/data/cts_lookup_table.json"
with open(lookup_path) as f:
    lookup = json.load(f)

In [4]:
lookup["urn:cts:greekLit:tlg0026.tlg003"]

['demosthenes_1_1863',
 'demosthenes_1856_1',
 'demosthenes_aeschynes_1861',
 'demosthenes_2_1861']

In [5]:
fr_embed_dir = "/scratch/craig.car/sents_embeds/fr_embeds/"
de_embed_dir = "/scratch/craig.car/sents_embeds/de_embeds/"
en_embed_dir = "/scratch/craig.car/sents_embeds/en_embeds/"
it_embed_dir = "/scratch/craig.car/sents_embeds/it_embeds/"
align_dir = "/home/craig.car/repos/chiron/chironata/data/alignments_rslts/"
text_dicts_dir = "/home/craig.car/repos/chiron/chironata/sentence_aligned_texts/"
eval_data_dir = "/home/craig.car/repos/chiron/chironata/eval_datafiles/"

dim = 768

src_embeds_dir = "/scratch/craig.car/sents_embeds/src_embeds/"
corpus_len = len([name for name in os.listdir(src_embeds_dir) if os.path.isfile(os.path.join(src_embeds_dir, name))])
print(corpus_len)

709


In [6]:
src_embeds_dir = "/scratch/craig.car/sents_embeds/src_embeds/"
corpus_len = len([name for name in os.listdir(src_embeds_dir) if os.path.isfile(os.path.join(src_embeds_dir, name))])
corpus_top1 = 0
corpus_top5 = 0
corpus_top10 = 0

for src_embed_path in ["/scratch/craig.car/sents_embeds/src_embeds/urn:cts:greekLit:tlg0019.tlg003.emb"]:
# for src_embed_path in glob.iglob(src_embeds_dir+"*.emb"):
    src_eval_data_dict = {}
    print(f"working on {src_embed_path}")
    text_top1 = 0
    text_top5 = 0
    text_top10 = 0
    # get embeds and resize
    src_embeddings = np.fromfile(src_embed_path, dtype=np.float32, count=-1)
    src_embeddings.resize(src_embeddings.shape[0] // dim, dim)
    # print(f"shape of src embeds {src_embeddings.shape}")
    # save num of src sents to src_eval data dict
    src_eval_data_dict["num_src_sents"] = src_embeddings.shape[0]
    # get translations
    ctsurn = os.path.splitext(os.path.basename(src_embed_path))[0]
    print(ctsurn)
    translations = lookup[ctsurn]
    print(translations)
    # for score denominators (reduce if encounter a translation with no rslts)
    num_transl = len(translations)
    # get text data dict
    text_json_path = text_dicts_dir+ctsurn+".json"
    with open(text_json_path) as f:
        text_json = json.load(f)
    for tgt_text in translations:
        if tgt_text.endswith(".xml"):
            tgt_text = tgt_text.split(".xml")[0]
        transl_eval_dict = {}
        print(f"working on {tgt_text} translation")
        # count scores per translation
        top_1 = 0
        top_5 = 0
        top_10 = 0
        num_nulls = 0
        # get embed file using lang in text_json
        if tgt_text not in text_json.keys():
            continue
        else:
            tgt_lang = text_json[tgt_text]["tgt_lang"]
            transl_eval_dict["tgt_lang"] = tgt_lang
            print(f"tgt lang is {tgt_lang}")
            # if os.path.isfile(de_embed_dir+tgt_text+".emb"):
            if tgt_lang == "de":
                tgt_embed = de_embed_dir+tgt_text+".emb"
            # elif os.path.isfile(en_embed_dir+tgt_text+".emb"):
            elif tgt_lang == "en":
                tgt_embed = en_embed_dir+tgt_text+".emb"
            # elif os.path.isfile(it_embed_dir+tgt_text+".emb"):
            elif tgt_lang == "it":
                tgt_embed = it_embed_dir+tgt_text+".emb"
            # elif os.path.isfile(fr_embed_dir+tgt_text+".emb"):
            elif tgt_lang == "fr":
                tgt_embed = fr_embed_dir+tgt_text+".emb"
            # get tgt embeds
            tgt_embeddings = np.fromfile(tgt_embed, dtype=np.float32, count=-1)
            tgt_embeddings.resize(tgt_embeddings.shape[0] // dim, dim)
            # print(f"shape of tgt embeds {tgt_embeddings.shape}")
            # build sim matrix
            sim_matrix = compute_sim_matrix_cosine(src_embeddings, tgt_embeddings)
            # iterate through src sents, compare sim score (src_sent,pred_sent) to top 1,5,10 scores
            for src_idx, src_sent_vector in enumerate(sim_matrix):
                # print(f"len of src sent vector is {len(src_sent_vector)}, src idx is {src_idx}")
                # skip translations that were deleted
                if tgt_text not in text_json.keys():
                    # back out from len of translations
                    num_transl -= 1
                else:
                    pred_sents_idx = text_json[tgt_text]["aligns_idx"][str(src_idx)]
                    # print(pred_sents_idx)
                    # sort vector of sim scores
                    sorted_sim_vec = np.flip(np.sort(src_sent_vector))
                    # check for each pred_sent in pred_sents
                    for tgt_sent_idx in pred_sents_idx:
                        if tgt_sent_idx == "null":
                            num_nulls += 1
                            continue
                        else:
                            tgt_sent_simscore = src_sent_vector[tgt_sent_idx]
                            if tgt_sent_simscore >= sorted_sim_vec[0]:
                                # print("have top 1")
                                top_1 += 1
                                top_5 += 1
                                top_10 += 1
                                continue
                            elif tgt_sent_simscore >= sorted_sim_vec[4]:
                                # print("have top 5")
                                top_5 += 1
                                top_10 += 1
                                continue    
                            elif tgt_sent_simscore >= sorted_sim_vec[9]:
                                top_10 += 1
                                # print("have top 10")
                                continue
        
        # how many src sents have a pred that's in top 1, top 5, top 10? Don't back out num_nulls from denominator bc no src sent should be aligned to null
        top_1_transl = top_1/sim_matrix.shape[0]
        top_5_transl = top_5/sim_matrix.shape[0]
        top_10_transl = top_10/sim_matrix.shape[0]
        print(f"top 10 is {top_10}")
        print(f"top 1 by transl is {top_1_transl}")
        print(f"top 5 by transl is {top_5_transl}")
        print(f"top 10 by transl is {top_10_transl}")
        transl_eval_dict["transl_top1"] = top_1_transl
        transl_eval_dict["transl_top5"] = top_5_transl
        transl_eval_dict["transl_top10"] = top_10_transl
        transl_eval_dict["num_nulls"] = num_nulls
        transl_eval_dict["num_tgt_sents"] = tgt_embeddings.shape[0]
        # add to src_eval_data_dict
        src_eval_data_dict[tgt_text] = transl_eval_dict
        
        # add to text-level average
        text_top1 += top_1_transl/num_transl
        text_top5 += top_5_transl/num_transl
        text_top10 += top_10_transl/num_transl
        print(f"text top 10 is {text_top10}")
        
    src_eval_data_dict["num_translations"] = num_transl
    src_eval_data_dict["text_top1"] = text_top1
    src_eval_data_dict["text_top5"] = text_top5
    src_eval_data_dict["text_top10"] = text_top10
    
    # write to json
    path_out = eval_data_dir+ctsurn+".json"
    with open(path_out, 'w') as fp:
        json.dump(src_eval_data_dict, fp, ensure_ascii=False)

    # add to corpus-level average
    corpus_top1 += text_top1/corpus_len
    corpus_top5 += text_top5/corpus_len
    corpus_top10 += text_top10/corpus_len

print(f"top1 corpus is {corpus_top1}")
print(f"top5 corpus is {corpus_top5}")
print(f"top10 corpus is {corpus_top10}")
        


working on /scratch/craig.car/sents_embeds/src_embeds/urn:cts:greekLit:tlg0019.tlg003.emb
urn:cts:greekLit:tlg0019.tlg003
['Aristophanes_1_1855-1885', 'aristophanes_1_1858', 'aristophanes_1_1915', 'aristophanes_2_1915', 'aristophanes_3_1830', 'aristophanes_1881_1', 'aristophanes_1_1830', 'greekDramas_1912', 'aristophanes_1873', 'tlg0019.tlg003.ogl-eng2.xml', 'Aristophanes_2_1855-1885', 'aristophanes_1907_1']
working on Aristophanes_1_1855-1885 translation
tgt lang is de
top 10 is 139
top 1 by transl is 0.059548254620123205
top 5 by transl is 0.09958932238193019
top 10 by transl is 0.14271047227926079
text top 10 is 0.011892539356605066
working on aristophanes_1_1858 translation
tgt lang is en
top 10 is 586
top 1 by transl is 0.34394250513347024
top 5 by transl is 0.5102669404517454
top 10 by transl is 0.6016427104722792
text top 10 is 0.06202943189596167
working on aristophanes_1_1915 translation
tgt lang is it
top 10 is 47
top 1 by transl is 0.006160164271047228
top 5 by transl is 0.0

In [7]:
src_eval_data_dict

{'num_src_sents': 974,
 'Aristophanes_1_1855-1885': {'tgt_lang': 'de',
  'transl_top1': 0.059548254620123205,
  'transl_top5': 0.09958932238193019,
  'transl_top10': 0.14271047227926079,
  'num_nulls': 0,
  'num_tgt_sents': 5841},
 'aristophanes_1_1858': {'tgt_lang': 'en',
  'transl_top1': 0.34394250513347024,
  'transl_top5': 0.5102669404517454,
  'transl_top10': 0.6016427104722792,
  'num_nulls': 0,
  'num_tgt_sents': 10625},
 'aristophanes_1_1915': {'tgt_lang': 'it',
  'transl_top1': 0.006160164271047228,
  'transl_top5': 0.027720739219712527,
  'transl_top10': 0.048254620123203286,
  'num_nulls': 0,
  'num_tgt_sents': 2665},
 'aristophanes_2_1915': {'tgt_lang': 'it',
  'transl_top1': 0.08521560574948665,
  'transl_top5': 0.1540041067761807,
  'transl_top10': 0.1868583162217659,
  'num_nulls': 2,
  'num_tgt_sents': 3069},
 'aristophanes_3_1830': {'tgt_lang': 'fr',
  'transl_top1': 0.004106776180698152,
  'transl_top5': 0.026694045174537988,
  'transl_top10': 0.053388090349075976,
  

In [27]:
print(f"top1 corpus is {corpus_top1}")
print(f"top5 corpus is {corpus_top5}")
print(f"top10 corpus is {corpus_top10}")

top1 corpus is 0.21721537007984332
top5 corpus is 0.3277851981538398
top10 corpus is 0.38836388697782265
